# **0.2 Construcción del Dataset**

In [2]:
# Importamos librerías 
import pandas as pd
import numpy as np
import os

In [3]:
# Cargar datos 
data_dir = os.path.join("..","data","processed","raw_cleaned")

ppi_c = pd.read_csv(os.path.join(data_dir, "ppi_clean.csv"))
target_c = pd.read_csv(os.path.join(data_dir, "target_clean.csv"))
topo_c = pd.read_csv(os.path.join(data_dir, "topo_clean.csv"))
go_c = pd.read_csv(os.path.join(data_dir, "go_enrichment_clean.csv"))


Conjunto de proteínas en estudio P.

Procesar anotaciones GO (Construye G)

In [4]:
# Separar proteínas asociadas a cada término GO
go_expanded = go_c.copy()
go_expanded['Overlapping genes'] = go_expanded['Overlapping genes'].astype(str).str.split(',')
go_expanded = go_expanded.explode('Overlapping genes')
go_expanded['Overlapping genes'] = go_expanded['Overlapping genes'].str.strip()
go_expanded = go_expanded.rename(columns={'Overlapping genes': 'Protein'})

Matriz funcional X_GO. donde filas = proteinas y columnas = términos GO.

In [5]:
# Matriz funcional: 1 si proteína tiene GO, 0 si no
protein_go_matrix = pd.crosstab(go_expanded['Protein'], go_expanded['GO'])

Tratamiento para complejos proteicos. Consutruye participantes (p_i) de cada complejo proteico.

In [6]:
# Filtrar complejos proteicos y extraer proteínas participantes
complex_df = target_c[target_c["Target_type"] == "PROTEIN COMPLEX"].copy()
complex_df = complex_df[complex_df["Complex_participants"].notnull()]
complex_df["Complex_participants"] = complex_df["Complex_participants"].astype(str).str.split(",")
complex_df = complex_df.explode("Complex_participants")
complex_df["Complex_participants"] = complex_df["Complex_participants"].str.strip()


Asignar GO a complejos proteicos por herencia. Aqui se hace X_GO[p_i, p_j]

In [7]:
# Mapeo: participante → complejo
complex_map = complex_df.rename(columns={"Complex_participants": "Protein", "Node_id": "Complex_id"})
complex_map = complex_map[["Protein", "Complex_id"]]

# Herencia de GO: participante → complejo
go_with_complex = protein_go_matrix.reset_index().merge(complex_map, on="Protein", how="inner")
go_by_complex = go_with_complex.drop(columns=["Protein"]).groupby("Complex_id").max()


Unir proteinas individuales y complejas en una sola matriz. Aqui se construye P.

In [8]:
# Extraer proteínas individuales
single_proteins = target_c[target_c["Target_type"] == "SINGLE PROTEIN"]["Node_id"]
go_single = protein_go_matrix.loc[protein_go_matrix.index.isin(single_proteins)].copy()
go_single.index.name = "Node_id"

# Renombrar índices para unir
go_by_complex.index.name = "Node_id"

# Unión final
X_GO = pd.concat([go_single, go_by_complex], axis=0)


In [9]:
# cuantas single_protein hay en X_GO
print(f"Total single proteins in X_GO: {go_single.shape[0]}")
# cuantos complejos hay en X_GO
print(f"Total complexes in X_GO: {go_by_complex.shape[0]}")

Total single proteins in X_GO: 3753
Total complexes in X_GO: 1408


In [10]:
# Matriz X.
print(X_GO.shape)  # (5161, 6928)

(5161, 6928)


In [18]:
# Guardamos 
output_dir = os.path.join("..","data","processed", "raw_cleaned")

dataset_completo = X_GO.reset_index()

# Guardar como CSV
dataset_completo.to_csv(os.path.join(output_dir, 'dataset_completo.csv'), index=False)

# Verificación
print("\nDatos guardados correctamente.")
print(f"Archivo guardado en: {os.path.join(output_dir, 'dataset_completo.csv')}")


Datos guardados correctamente.
Archivo guardado en: ..\data\processed\raw_cleaned\dataset_completo.csv


In [19]:
# Visualización dataset
print("\nPrimeras filas del dataset completo:")
print(dataset_completo.head())


Primeras filas del dataset completo:
  Node_id  GO:0000002_BP  GO:0000012_BP  GO:0000014_MF  GO:0000018_BP  \
0   A2ML1              0              0              0              0   
1    AAK1              0              0              0              0   
2   AARS1              0              0              0              0   
3   AARS2              0              0              0              0   
4    AASS              0              0              0              0   

   GO:0000022_BP  GO:0000027_BP  GO:0000028_BP  GO:0000030_MF  GO:0000038_BP  \
0              0              0              0              0              0   
1              0              0              0              0              0   
2              0              0              0              0              0   
3              0              0              0              0              0   
4              0              0              0              0              0   

   ...  GO:2001244_BP  GO:2001251_BP  GO:2

In [14]:
# Ver cuantos complejos y cuantas single_protein hay
print(f"Total single proteins in dataset: {dataset_completo[dataset_completo['Node_id'].isin(single_proteins)].shape[0]}")
print(f"Total complexes in dataset: {dataset_completo[dataset_completo['Node_id'].isin(complex_map['Complex_id'])].shape[0]}")

Total single proteins in dataset: 3753
Total complexes in dataset: 1408
